In [1]:
from playwright.async_api import async_playwright
import asyncio

In [2]:
"""
<td>
    <div class="obc-tmpl-character__voice-item obc-tmpl-character__play-voice">
        <div class="obc-tmpl-character__voice-btn">
        </div> 
        <span class="obc-tmpl-character__voice-content"> 	
            [text] 	 
        </span>
    </div>
 </td>
"""

'\n<td>\n    <div class="obc-tmpl-character__voice-item obc-tmpl-character__play-voice">\n        <div class="obc-tmpl-character__voice-btn">\n        </div> \n        <span class="obc-tmpl-character__voice-content"> \t\n            [text] \t \n        </span>\n    </div>\n </td>\n'

In [3]:
URL = "https://bbs.mihoyo.com/ys/obc/content/4073/detail"
TRAGET_FEATURE="https://act-upload.mihoyo.com/wiki-user-upload"
OUTPUT_DIR = "/tmp/audio"

In [14]:
async def scrape_texts_and_urls():
    results = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context() 
        page = await context.new_page()
        
        # Intercept all requests and add a "Cache-Control: no-cache" header.
        await page.route("**/*", lambda route, request: route.continue_(
            headers={**request.headers, "Cache-Control": "no-cache"}
        ))
        
        # Navigate to the page and wait for the containers.
        await page.goto(URL)
        await page.wait_for_selector("div.obc-tmpl-character__voice-item")
        
        # Select all containers that include both the text and the voice button.
        containers = await page.query_selector_all("div.obc-tmpl-character__voice-item.obc-tmpl-character__play-voice")
        
        for index, container in enumerate(containers):
            # Extract the text.
            text_el = await container.query_selector("span.obc-tmpl-character__voice-content")
            text = (await text_el.text_content()).strip() if text_el else ""
            
            # Locate the voice button.
            button = await container.query_selector("div.obc-tmpl-character__voice-btn")
            if not button:
                print(f"Button not found in container {index}")
                results.append((text, None))
                continue
            
            # Ensure the container is visible.
            await container.scroll_into_view_if_needed()
            
            mp3_url = None
            while mp3_url is None:
                try:
                    # Wait for the .mp3 GET request triggered by clicking the button.
                    async with page.expect_request(
                        lambda request: request.method == "GET" and request.url.endswith(".mp3"),
                        timeout=5000
                    ) as request_info:
                        await button.click(timeout=5000, force=True)
                        # Brief pause to allow the request to be initiated.
                        await asyncio.sleep(0.5)
                    mp3_request = await request_info.value
                    mp3_url = mp3_request.url
                    print(f"Captured mp3 for container {index}: {mp3_url}")
                except Exception as e:
                    #print(f"Retrying for container {index} text '{text}' due to: {e}")
                    #await asyncio.sleep(1)
                    mp3_url=""
            
            results.append((text, mp3_url))
        
        await browser.close()
    return results

l=await scrape_texts_and_urls()
    


Task exception was never retrieved
future: <Task finished name='Task-8127' coro=<Waiter.reject_on_timeout.<locals>.reject() done, defined at /home/co/miniconda3/envs/3.12/lib/python3.12/site-packages/playwright/_impl/_waiter.py:83> exception=TargetClosedError(': Target page, context or browser has been closed')>
Traceback (most recent call last):
  File "/home/co/miniconda3/envs/3.12/lib/python3.12/site-packages/playwright/_impl/_waiter.py", line 85, in reject
    self._reject(TimeoutError(message))
  File "/home/co/miniconda3/envs/3.12/lib/python3.12/site-packages/playwright/_impl/_waiter.py", line 103, in _reject
    self._cleanup()
  File "/home/co/miniconda3/envs/3.12/lib/python3.12/site-packages/playwright/_impl/_waiter.py", line 94, in _cleanup
    listener[0].remove_listener(listener[1], listener[2])
  File "/home/co/miniconda3/envs/3.12/lib/python3.12/site-packages/playwright/_impl/_connection.py", line 187, in remove_listener
    self._update_subscription(event, False)
  File 

Captured mp3 for container 0: https://act-upload.mihoyo.com/wiki-user-upload/2024/11/28/16576950/ad09f370287b5a366634391e4d93b47b_1148587448586586018.mp3
Captured mp3 for container 1: https://act-upload.mihoyo.com/wiki-user-upload/2024/11/28/16576950/a4f4b4b94ad01fb3086cd4630e92bfed_2996685260629673957.mp3
Captured mp3 for container 2: https://act-upload.mihoyo.com/wiki-user-upload/2024/11/28/16576950/4a356ea48a7bfb1c9c73f9e8ba518bba_5847299019060754831.mp3
Captured mp3 for container 5: https://act-upload.mihoyo.com/wiki-user-upload/2024/11/28/16576950/95d9e221c80da5808db044a242415c83_4387121457163090170.mp3
Captured mp3 for container 6: https://act-upload.mihoyo.com/wiki-user-upload/2024/11/28/16576950/4567e9e66854c8399a740338106a1dc9_6069001359278851527.mp3
Captured mp3 for container 7: https://act-upload.mihoyo.com/wiki-user-upload/2024/11/28/16576950/de4ffa55220174bbcf59d0e55fa238d7_8630295430883601429.mp3
Captured mp3 for container 8: https://act-upload.mihoyo.com/wiki-user-upload

In [15]:
l[:5]

[('•派蒙：蒙德有很多风车呢。•玩家：因为蒙德城建在水平面的上方，所以是靠风车把水抽上来的吧。•派蒙：回答正确！蒙德四季风吹不断，所以水源的供应也很稳定。•派蒙：而且，风车就是所谓「看得见的风」吧——然后，风铃就是「听得见的风」。•派蒙：也算是一种祈求风神护佑的吉祥物啦。•玩家：原来如此，和派蒙一样是吉祥物啊。•派蒙：不一样！风车是用木头做的，不能当应急食品，只能用来磨牙啊！•玩家：嗯…•玩家：…我该接话吗？',
  'https://act-upload.mihoyo.com/wiki-user-upload/2024/11/28/16576950/ad09f370287b5a366634391e4d93b47b_1148587448586586018.mp3'),
 ('•玩家：我听说，收了可观的小费以后，酒馆老板会小声说出一些情报。•派蒙：是「天使的馈赠」吗？但那位迪卢克老爷，好像是非常有钱的人。•派蒙：究竟要多少小费，才能让老板觉得「可观」呢？•玩家：唔…那也许是我记错了吧。•玩家：也许这个传言，是在别的世界听到的…•派蒙：你去过的世界真的很多呢。',
  'https://act-upload.mihoyo.com/wiki-user-upload/2024/11/28/16576950/a4f4b4b94ad01fb3086cd4630e92bfed_2996685260629673957.mp3'),
 ('•派蒙：果酒湖是不冻湖，一整年里都不会结冰。•派蒙：蒙德最好的苹果酒，只能用这座湖里的甜水来酿。•派蒙：湖水真的很好喝！我一秒就能喝掉两大瓶。•玩家：真是惊人。•派蒙：哼哼。•玩家：派蒙这样的体型，喝这么多水…•玩家：居然不会像风史莱姆那样，炸到漏气满天飞…•派蒙：哼哼，装好喝的饮料的是另一个胃。•玩家：嗯…世界真是广大，需要花时间去思考的见闻，还有很多…',
  'https://act-upload.mihoyo.com/wiki-user-upload/2024/11/28/16576950/4a356ea48a7bfb1c9c73f9e8ba518bba_5847299019060754831.mp3'),
 ('•玩家：丽莎提到过的，那些「禁书区」的书…•玩家：你有没有好奇过？什么样的书会被放在禁书区里呢？•玩家：莫非…•玩家：会有那种

In [21]:
import os
import requests
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [29]:
quarter = len(l) // 4
marks=["C","J","E","K"]

In [31]:
with open(os.path.join(OUTPUT_DIR,f"index.csv"),"w") as f:
    i=0
    while i<quarter:
        f.write(f"{i},{l[i][0]}\n")
        i+=1


In [33]:
# Headers from network inspection
HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:136.0) Gecko/20100101 Firefox/136.0",
    "Referer": "https://bbs.mihoyo.com/",
    "Accept": "audio/webm,audio/ogg,audio/wav,audio/*;q=0.9,application/ogg;q=0.7,video/*;q=0.6,*/*;q=0.5",
    "DNT": "1",
    "Sec-Fetch-Dest": "audio",
    "Sec-Fetch-Mode": "no-cors",
    "Sec-Fetch-Site": "same-site"
}

In [36]:
person_id=URL.split('/')[-2]
person_id

'4073'

In [35]:
for i, (text, url) in enumerate(l):
    if url:
        q = i // quarter
        response = requests.get(url, headers=HEADERS, stream=True)
        response.raise_for_status()
        with open(os.path.join(OUTPUT_DIR,person_id,marks[q],f"({i-q*quarter}.mp3"),"wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:  # Filter out keep-alive chunks
                    f.write(chunk)

